In [32]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine,MetaData,Table,select

In [2]:
url = "https://money.cnn.com/data/hotstocks/"
#headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"} 

In [3]:
res = requests.get(url)

In [4]:
soup = BeautifulSoup(res.content,"html.parser")

In [5]:
soup.title.text

'Hot Stocks - Stock Market Movers - CNNMoney'

In [6]:
b=soup.select( ".wsod_symbol" )

In [7]:
a=soup.find("table",class_= "wsod_dataTable wsod_dataTableBigAlt" )


In [8]:
c=a.select( ".wsod_symbol" )

In [9]:
symbols=[]
for items in c:
    symbols.append(items.text)

In [10]:
asymbol=symbols

In [11]:
asymbol

['BAC', 'CCL', 'F', 'KO', 'TFC', 'T', 'VZ', 'PCG', 'C', 'NEE']

In [12]:
result =[]

In [13]:
for items in asymbol:
    url="https://finance.yahoo.com/quote/{}".format(items)
    req=requests.get(url.format('1'))
    ysoup = BeautifulSoup(req.text,"html.parser")
    
    #print(items)
    
    name = ysoup.find('h1',class_='D(ib) Fz(18px)')
    na=name.text
    print(na)
    
    tab1=ysoup.find('table',{'class':"W(100%)"})
    se=tab1.find('td',{'data-test': 'OPEN-value'})
    op=se.text
    print("Open:-",op)
    
    #tab1=ysoup.find('table',{'class':"W(100%)"})
    volume=tab1.find('td',{'data-test': 'TD_VOLUME-value'})
    vol=volume.text
    print("volume:-",vol)
    
    tab2=ysoup.find('table',{'class':"W(100%) M(0) Bdcl(c)"})
    per=tab1.find('td',{'data-test': 'PE_RATIO-value'})
    pe=se.text
    print("PERatio:-",pe)
    print("------------")
    
    result.append([na,op,vol,pe])

     

Bank of America Corporation (BAC)
Open:- 26.61
volume:- 14,305,382
PERatio:- 26.61
------------
Carnival Corporation & plc (CCL)
Open:- 12.82
volume:- 9,312,154
PERatio:- 12.82
------------
Ford Motor Company (F)
Open:- 12.10
volume:- 6,844,379
PERatio:- 12.10
------------
The Coca-Cola Company (KO)
Open:- 54.13
volume:- 7,210,228
PERatio:- 54.13
------------
Truist Financial Corporation (TFC)
Open:- 29.37
volume:- 6,682,820
PERatio:- 29.37
------------
AT&T Inc. (T)
Open:- 14.69
volume:- 6,572,812
PERatio:- 14.69
------------
Verizon Communications Inc. (VZ)
Open:- 31.48
volume:- 4,861,392
PERatio:- 31.48
------------
PG&E Corporation (PCG)
Open:- 15.62
volume:- 4,779,819
PERatio:- 15.62
------------
Citigroup Inc. (C)
Open:- 41.14
volume:- 4,115,110
PERatio:- 41.14
------------
NextEra Energy, Inc. (NEE)
Open:- 49.68
volume:- 4,231,435
PERatio:- 49.68
------------


In [14]:
print(result)

[['Bank of America Corporation (BAC)', '26.61', '14,305,382', '26.61'], ['Carnival Corporation & plc (CCL)', '12.82', '9,312,154', '12.82'], ['Ford Motor Company (F)', '12.10', '6,844,379', '12.10'], ['The Coca-Cola Company (KO)', '54.13', '7,210,228', '54.13'], ['Truist Financial Corporation (TFC)', '29.37', '6,682,820', '29.37'], ['AT&T Inc. (T)', '14.69', '6,572,812', '14.69'], ['Verizon Communications Inc. (VZ)', '31.48', '4,861,392', '31.48'], ['PG&E Corporation (PCG)', '15.62', '4,779,819', '15.62'], ['Citigroup Inc. (C)', '41.14', '4,115,110', '41.14'], ['NextEra Energy, Inc. (NEE)', '49.68', '4,231,435', '49.68']]


In [15]:
colname=['Company','Open_value','Volume','PE_Ratio']
df = pd.DataFrame(data=result,columns=colname)

In [16]:
df

,Company,Open_value,Volume,PE_Ratio
0,Bank of America Corporation (BAC),26.61,"14,305,382",26.61
1,Carnival Corporation & plc (CCL),12.82,"9,312,154",12.82
2,Ford Motor Company (F),12.10,"6,844,379",12.10
3,The Coca-Cola Company (KO),54.13,"7,210,228",54.13
4,Truist Financial Corporation (TFC),29.37,"6,682,820",29.37
5,AT&T Inc. (T),14.69,"6,572,812",14.69
6,Verizon Communications Inc. (VZ),31.48,"4,861,392",31.48
7,PG&E Corporation (PCG),15.62,"4,779,819",15.62
8,Citigroup Inc. (C),41.14,"4,115,110",41.14
9,"NextEra Energy, Inc. (NEE)",49.68,"4,231,435",49.68


In [17]:
df.to_csv('project1_2.csv',mode='w')   #exported to csv 

In [18]:
df['Open_value'] = df['Open_value'].astype(str).astype(float)
#df['Volume'] = df['Volume'].replace(',','').astype(int)
df['PE_Ratio'] = df['PE_Ratio'].astype(str).astype(float)


In [19]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Company     10 non-null     object 
 1   Open_value  10 non-null     float64
 2   Volume      10 non-null     object 
 3   PE_Ratio    10 non-null     float64
dtypes: float64(2), object(2)
memory usage: 452.0+ bytes


In [20]:
dbase1 = create_engine('sqlite:///dbase1.db')

In [21]:
df.to_sql(name='stocks',con=dbase1,if_exists='replace', index=False)

10

In [22]:
pd.read_sql(sql='stocks',con=dbase1)

,Company,Open_value,Volume,PE_Ratio
0,Bank of America Corporation (BAC),26.61,"14,305,382",26.61
1,Carnival Corporation & plc (CCL),12.82,"9,312,154",12.82
2,Ford Motor Company (F),12.10,"6,844,379",12.10
3,The Coca-Cola Company (KO),54.13,"7,210,228",54.13
4,Truist Financial Corporation (TFC),29.37,"6,682,820",29.37
5,AT&T Inc. (T),14.69,"6,572,812",14.69
6,Verizon Communications Inc. (VZ),31.48,"4,861,392",31.48
7,PG&E Corporation (PCG),15.62,"4,779,819",15.62
8,Citigroup Inc. (C),41.14,"4,115,110",41.14
9,"NextEra Energy, Inc. (NEE)",49.68,"4,231,435",49.68


In [23]:
sqlalchemy.__version__

'1.4.39'

# get a column "company"

In [24]:
table_name = 'stocks'  
column_name = 'Company'

In [25]:
print(f"The column '{column_name}' exists in the table '{table_name}'.")

The column 'Company' exists in the table 'stocks'.


# 

In [26]:
metadata = MetaData()

In [27]:
table = Table(table_name, metadata, autoload=True, autoload_with=dbase1)

In [28]:
connection = dbase1.connect()

In [29]:
select_stmt = table.select().with_only_columns([table.c[column_name]])
result = connection.execute(select_stmt)

In [30]:
for row in result:
    print(row[column_name])

# Close the database connection
connection.close()

Bank of America Corporation (BAC)
Carnival Corporation & plc (CCL)
Ford Motor Company (F)
The Coca-Cola Company (KO)
Truist Financial Corporation (TFC)
AT&T Inc. (T)
Verizon Communications Inc. (VZ)
PG&E Corporation (PCG)
Citigroup Inc. (C)
NextEra Energy, Inc. (NEE)


# check an element

In [33]:
table = Table('stocks', metadata, autoload=True, autoload_with=dbase1)
element_to_find = 'Ford Motor Company (F)'

# Create a query to fetch a row based on a condition
query = select([table]).where(table.c.Company == element_to_find)

# Execute the query
result = dbase1.execute(query)

# Fetch the row
row = result.fetchone()

if row:
    print("Row found:")
    print(row)
else:
    print("Row not found")

# Close the engine when done
#engine.dispose()

Row found:
('Ford Motor Company (F)', 12.1, '6,844,379', 12.1)
